In [2]:
# enter the machine and conda activate
import shutil
import os
import sys
import subprocess

import fileinput


before run the workflow we need this files in working directory:

    POSCAR_BZO_ROTATION
    in_template.lammps
    graph18-compress0.pb
    dump2xml.py

In [1]:
result = subprocess.run(["phono3py", "-d", "--dim=\"1 1 1\"", "-c", "POSCAR_BZO_ROTATION"], capture_output=True, text=True) # phono3py -d --dim="1 1 1" -c POSCAR_BZO_ROTATION 

print(result.stdout)

NameError: name 'subprocess' is not defined

In [ ]:
# mv the POSCAR-00xxx to POSCAR-OO directory
if os.path.exists('POSCAR-OO'): 
    shutil.rmtree('POSCAR-OO')
os.mkdir("POSCAR-OO")
shutil.move("POSCAR-00*", "POSCAR-OO/POSCAR-00*")       # the wildcard is enable?

In [ ]:
# format poscar to lmp for deepmd lmp input
if os.path.exists('lmp'): 
    shutil.rmtree('lmp')
os.mkdir("lmp")

for p, d, files in os.walk("POSCAR-OO"):
    for f in files:
        index_name = f.split('.')[0]
        subprocess.run(["atomsk", "POSCAR-OO/"+f, "lmp/", "lammps"])      # atomsk POSCAR-00/POSCAR-00001 lammps


In [10]:
# in_f = 'in_00002.lammps'
for p, d, files in os.walk('lmp'):
    for f in files:
        if f.endswith('.lmp'):
            base_name = 'POSCAR-00'
            index = f.split('.')[0].replace(base_name, '')
            # index = '001'
            try:
                shutil.copy("in_template.lammps", f"lmp/in-00{index}.lammps")
            except:
                print("in_template.lammps not found")


In [11]:
# generate in.lammps for deepmd
for p, d, files in os.walk("lmp"):
    for f in files:
        if f.endswith('.lammps'):
            base_name='in-00'
            index = f.split('.')[0].replace(base_name, '')
            print(base_name, f, index)
            # configure read_data and write_dump 
            import fileinput
            old_tag = "<index>"
            new_tag = index
            with fileinput.input(os.path.join('lmp', f), inplace=True) as f_handle:
                for line in f_handle:
                    print(line.replace(old_tag, new_tag), end='')
                    

in-00 in-00002.lammps 002


In [18]:
# this is test sychronize in python,wait my_func() and then excute the below code.

from time import sleep
import threading

def my_func():
    sleep(10)
    print(
        "I'm working on something that takes longer than 10 seconds."
    )

t = threading.Thread(target=my_func)
t.start()

sleep(2)   
print("I'm tired of waiting.before join, i sleep 2 seconds.")

t.join()

print("I finished work before join.")
# the result should like this sequence:
# print("I'm tired of waiting.before join, i sleep 2 seconds.")
# print("I'm working on something that takes longer than 10 seconds.")
# print("I finished work before join.")


I'm tired of waiting.before join, i sleep 2 seconds.
I'm working on something that takes longer than 10 seconds.
I finished work before join.


In [17]:
# run deepmd lmp in lmp files , run this command is need wait for a while time. count
import timeit
import threading    

def mpi_run():
    start_time = timeit.default_timer()
    
    count = 0
    for p, d, files in os.walk("lmp"):
        for f in files:
            if f.endswith('.lammps'):
                count += 1
                index = f.split('.')[0].replace('in-', '')
                # TODO THIS COMMAND CAN SYCHRONIZE ???
                p = subprocess.Popen(["mpirun", "-n", "10", "lmp", "-in", f"lmp/{f}", f">lmp-{index}.out", f"2>lmp-{index}.err"])# mpirun -n 10 lmp -in in-00002.lammps >lmp-00002.out 2>lmp-00002.err
                p.wait()
    
    end_time = timeit.default_timer()
    print('Running time: %d Seconds.' % (end_time - start_time))
    if count != 0:
        print('Executed %s times per sample.' % count)

t = threading.Thread(target=mpi_run)
t.start()
t.join()

Running time: 1 Seconds
Executed 1 times per sample 1


In [21]:
# format deepmd output BZO-00002.dump to xml file for phono3py
def read_dump(dump_file):
    with open(dump_file, 'r') as f:
        lines = f.readlines()
    lines = lines[9:]
    data = [i.split() for i in lines]
    fxfyfz = []
    for i in data:
        fxfyfz.append([float(i[-3]), float(i[-2]), float(i[-1])])
    return fxfyfz

def write_xml(fxfyfz, xml_file):
    with open(xml_file, 'w') as f:
        f.write('<?xml version="1.0" encoding="ISO-8859-1"?>\n')
        f.write('<modeling>\n')
        f.write('   <calculation>\n')
        f.write('       <varray name="force">\n')
        # the force data is in the 10th line to the end
        for i in fxfyfz:
            f.write(f'           <v>{i[0]} {i[1]} {i[2]}</v>\n')
        f.write('       </varray>\n')
        f.write('   </calculation>\n')
        f.write('</modeling>\n')

In [22]:
for p, d, files in os.walk("output"):
    for f in files:
        if f.endswith('.dump'):
            dump_file = os.path.join('output', f)
            xml_file = os.path.join('output', f.replace('.dump', '.xml'))
            fxfyfz = read_dump(dump_file)
            write_xml(fxfyfz, xml_file)

In [ ]:
# use phono3py generate FORCE_FC3 and thermal conductivity
import subprocess

command_1 = subprocess.Popen(["phono3py", "--cf3", "output/*.xml"])
command_1.wait()
command_2 = subprocess.Popen(["phono3py", "--sym-fc"])
command_2.wait()
command_3 = subprocess.run(["phono3py", "--fc3", "--fc2", "--dim=\"1 1 1\"", "--mesh=\"10 10 10\"", "--br", "--tmin=10", "--tmax=1000", "&> Heat.txt"])
command_3.wait()

In [23]:
import subprocess

print('hello world')
p = subprocess.Popen(["sleep", "5"])
print('before 5 seconds')
p.wait()
print('after 5 seconds')

hello world
before 5 seconds
after 5 seconds
